**Lab 2: Advanced Linear Programming and Sensitivity Analysis**
- Defining more complex linear programming tasks
- Learning different constraint types
- Sensitivity analysis

**Production Optimization Problem**

This program solves a linear programming problem for optimizing production of three products (A, B, and C).

The A can be for example a number of beds, B can be meters of plywood and C can be meters of low quality plywood.
Notice that it does not make sense to produce a half of a bed, so we need to use integer variables for this decision variable.

**Decision Variables:**
- unitsA: Number of units of product A to produce (integer):
- unitsB: Number of units of product B to produce
- unitsC: Number of units of product C to produce

**Objective Function:**
- Maximize profit: 400 PLN per unit A + 300 PLN per unit B + 200 PLN per unit C

**Constraints:**
- Assembly time: 0.3h per A + 0.1h per B + 0.1h per C ≤ 1800 hours
- Quality control: 0.1h per A + 0.08h per B + 0.04h per C ≤ 800 hours
- Packaging: 0.06h per A + 0.04h per B + 0.05h per C ≤ 700 hours


In [ ]:
# In Google Colab, ensure PuLP is installed:
!pip install pulp

from pulp import (
    LpProblem,
    LpVariable,
    LpMaximize,
    LpInteger,
    LpContinuous,
    LpBinary,
    value,
    PULP_CBC_CMD
)

# 1) Create the optimization problem (maximize profit).
prob = LpProblem("Advanced_Production_Problem", LpMaximize)

# 2) Define Decision Variables
# Let's say:
#   - A (number of units of product A) is integer (like beds).
#   - B (number of units of product B) is continuous or integer, depending on your scenario.
#   - C (number of units of product C) is continuous or integer, too.

A = LpVariable("A", lowBound=0, cat=LpInteger)  # must be integer
B = LpVariable("B", lowBound=0)  # continous meters of plywood
C = LpVariable("C", lowBound=0)  # continous meters of low quality plywood

# 3) Define Objective Function
# Profit values (you can tweak these):
profit_A = 400
profit_B = 300
profit_C = 200

prob += profit_A*A + profit_B*B + profit_C*C, "Profit_Objective"

# 4) Define Constraints

# --- Resource / Time Constraints (same as the previous example, extended if desired) ---
# Example: max available hours in Assembly, Quality Control, and Packaging
prob += 0.3*A + 0.1*B + 0.1*C <= 1800, "Assembly_Hours"
prob += 0.1*A + 0.08*B + 0.04*C <= 800, "Quality_Control_Hours"
prob += 0.06*A + 0.04*B + 0.05*C <= 700, "Packaging_Hours"

# --- Minimum Demand Constraints ---
# Suppose the company must produce at least 100 units of A, 50 of B, and 80 of C to satisfy orders.
prob += A >= 100, "Min_Demand_A"
prob += B >= 50,  "Min_Demand_B"
prob += C >= 80,  "Min_Demand_C"

# --- Optional Additional Constraints ---
# For instance, if product C requires a special component that is limited:
# Let's say we have only 500 units of that component, and each unit of C consumes 1 unit of that component
# prob += C <= 500, "Special_Component_Limit"

# Alternatively, we might have a ratio constraint, e.g., for product mix synergy:
# For example, we do not want to produce more B than 2 times A
# prob += B <= 2 * A, "Mix_Ratio_Constraint"

# 5) Solve the problem
prob.writeLP("AdvancedProduction.lp")
prob.solve(PULP_CBC_CMD(msg=False))  # CBC solver

# 6) Print results
print("Status:", prob.status)
for v in prob.variables():
    print(v.name, "=", v.varValue)

print("Total profit = ", value(prob.objective))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 93.9 MB/s eta 0:00:00
Status: 1
A = 1894.0
B = 2948.1667
C = 9368.6667
Total profit =  3515783.3499999996


## 2. Interpreting the Extended Model



### Minimum Demand Constraints:
- E.g. `A >= 100` ensures at least 100 units of A are produced.

### Optional Constraints:
- Resource constraints, ratio constraints, or any other real-world limitations.



## 3. Performing Sensitivity Analysis

### Approach A: Manual Parameter Variation
- **Change the availability of resources:**
  - For example, reduce the 1800 hours of Assembly to 1500, solve again, and observe the new optimal solution.
- **Change the profit coefficients:**
  - If the profit for product C increases to 250, does the solution shift toward more C?
- **Change the minimum demand:**
  - If the market demands 150 units of A instead of 100, how does that affect the objective?

## Exercise 1: Minimum Demand and New Constraints

- Implement the code above and check if it finds a feasible solution.
- Alter the minimum demands:
  - Increase or decrease them to see if the solution changes drastically.
- Interpret which constraints become "binding" (fully used, the value of the constraint is equal to its limit) in the optimal solution.

In [ ]:
!pip install pulp

from pulp import (
    LpProblem,
    LpVariable,
    LpMaximize,
    LpInteger,
    LpContinuous,
    LpBinary,
    value,
    PULP_CBC_CMD
)

# 1) Create the optimization problem (maximize profit).
prob = LpProblem("Advanced_Production_Problem", LpMaximize)

# 2) Define Decision Variables
# Let's say:
#   - A (number of units of product A) is integer (like beds).
#   - B (number of units of product B) is continuous or integer, depending on your scenario.
#   - C (number of units of product C) is continuous or integer, too.

A = LpVariable("A", lowBound=0, cat=LpInteger)  # must be integer
B = LpVariable("B", lowBound=0)  # continous meters of plywood
C = LpVariable("C", lowBound=0)  # continous meters of low quality plywood

# 3) Define Objective Function
# Profit values (you can tweak these):
profit_A = 400
profit_B = 300
profit_C = 200

prob += profit_A*A + profit_B*B + profit_C*C, "Profit_Objective"

# 4) Define Constraints

# --- Resource / Time Constraints (same as the previous example, extended if desired) ---
# Example: max available hours in Assembly, Quality Control, and Packaging
prob += 0.3*A + 0.1*B + 0.1*C <= 1800, "Assembly_Hours"
prob += 0.1*A + 0.08*B + 0.04*C <= 800, "Quality_Control_Hours"
prob += 0.06*A + 0.04*B + 0.05*C <= 700, "Packaging_Hours"

# --- Minimum Demand Constraints ---
# Suppose the company must produce at least 100 units of A, 50 of B, and 80 of C to satisfy orders.
prob += A >= 100, "Min_Demand_A"
prob += B >= 50,  "Min_Demand_B"
prob += C >= 80,  "Min_Demand_C"

# --- Optional Additional Constraints ---
# For instance, if product C requires a special component that is limited:
# Let's say we have only 500 units of that component, and each unit of C consumes 1 unit of that component
# prob += C <= 500, "Special_Component_Limit"

# Alternatively, we might have a ratio constraint, e.g., for product mix synergy:
# For example, we do not want to produce more B than 2 times A
# prob += B <= 2 * A, "Mix_Ratio_Constraint"

# 5) Solve the problem
prob.writeLP("AdvancedProduction.lp")
prob.solve(PULP_CBC_CMD(msg=False))  # CBC solver

# 6) Print results
print("Status:", prob.status)
for v in prob.variables():
    print(v.name, "=", v.varValue)

print("Total profit = ", value(prob.objective))

Status: 1
A = 1894.0
B = 2948.1667
C = 9368.6667
Total profit =  3515783.3499999996


## Exercise 2: Sensitivity Analysis on Resource Availability

- Create a loop that iterates over possible Assembly hours: 1600, 1800, 2000.
- For each iteration, solve the problem and record:
  - The optimal quantity of A, B, C.
  - The total profit.
- Plot or tabulate results to see the trend (if you like, e.g., in a DataFrame).

In [ ]:
asmhours = [1600, 1800, 2000]

for i in asmhours:
  prob = LpProblem("Advanced_Production_Problem", LpMaximize)

  A = LpVariable("A", lowBound=0, cat=LpInteger)  # must be integer
  B = LpVariable("B", lowBound=0)  # continous meters of plywood
  C = LpVariable("C", lowBound=0)  # continous meters of low quality plywood
  profit_A = 400
  profit_B = 300
  profit_C = 200

  prob += profit_A*A + profit_B*B + profit_C*C, "Profit_Objective"
  prob += 0.3*A + 0.1*B + 0.1*C <= i, "Assembly_Hours"
  prob += 0.1*A + 0.08*B + 0.04*C <= 800, "Quality_Control_Hours"
  prob += 0.06*A + 0.04*B + 0.05*C <= 700, "Packaging_Hours"
  prob += A >= 100, "Min_Demand_A"
  prob += B >= 50,  "Min_Demand_B"
  prob += C >= 80,  "Min_Demand_C"

  prob.writeLP("AdvancedProduction.lp")
  prob.solve(PULP_CBC_CMD(msg=False))  # CBC solver

  print("Status:", prob.status)
  for v in prob.variables():
      print(v.name, "=", v.varValue)

  print("Total profit = ", value(prob.objective))

Status: 1
A = 631.0
B = 4316.4167
C = 9789.6667
Total profit =  3505258.3499999996
Status: 1
A = 1894.0
B = 2948.1667
C = 9368.6667
Total profit =  3515783.3499999996
Status: 1
A = 3157.0
B = 1579.9167
C = 8947.6667
Total profit =  3526308.3499999996


## Exercise 3 (Optional): Binary Decision Constraints

- Add a binary variable that indicates whether you open a specific production line (1) or not (0).
- If that line is closed, the hours available might be reduced or zero.
- Solve and see how the solver decides the best strategy (to open or not to open).

In [ ]:
is_line_open_a = LpVariable("is_line_open_a", cat=LpBinary)
is_line_open_b = LpVariable("is_line_open_b", cat=LpBinary)
is_line_open_c = LpVariable("is_line_open_c", cat=LpBinary)

prob = LpProblem("Advanced_Production_Problem", LpMaximize)

A = LpVariable("A", lowBound=0, cat=LpInteger)  # must be integer
B = LpVariable("B", lowBound=0)  # continous meters of plywood
C = LpVariable("C", lowBound=0)  # continous meters of low quality plywood
profit_A = 400
profit_B = 300
profit_C = 200

prob += profit_A*A + profit_B*B + profit_C*C, "Profit_Objective"

# Define a large constant (Big M)
M = 10000 # Choose a sufficiently large number

# --- Resource / Time Constraints with binary variables ---
# If is_line_open_a is 0, then A must be 0 (or very small if lowBound is not 0)
# If is_line_open_a is 1, this constraint is effectively removed
prob += 0.3*A <= is_line_open_a * M, "Assembly_A_limit"
prob += 0.1*B <= is_line_open_b * M, "Assembly_B_limit"
prob += 0.1*C <= is_line_open_c * M, "Assembly_C_limit"

# Now add the original resource constraints, but only if the line is open
# This requires that if a line is closed (binary variable is 0), then the production of that product is 0
prob += 0.3*A + 0.1*B + 0.1*C <= 1800, "Assembly_Hours"
prob += 0.1*A + 0.08*B + 0.04*C <= 800, "Quality_Control_Hours"
prob += 0.06*A + 0.04*B + 0.05*C <= 700, "Packaging_Hours"

# --- Minimum Demand Constraints ---
# These still apply regardless of the line being open or not, but the solver will need to find a feasible solution
# considering the binary constraints. You might need to adjust these if closing a line makes meeting demand impossible.
prob += A >= 100 * is_line_open_a, "Min_Demand_A" # Demand applies only if line is open
prob += B >= 50 * is_line_open_b,  "Min_Demand_B"
prob += C >= 80 * is_line_open_c,  "Min_Demand_C"


# 5) Solve the problem
prob.writeLP("AdvancedProduction.lp")
prob.solve(PULP_CBC_CMD(msg=False))  # CBC solver

# 6) Print results
print("Status:", prob.status)
for v in prob.variables():
    print(v.name, "=", v.varValue)

print("Total profit = ", value(prob.objective))

Status: 1
A = 1894.0
B = 2948.1667
C = 9368.6667
is_line_open_a = 1.0
is_line_open_b = 1.0
is_line_open_c = 1.0
Total profit =  3515783.3499999996


# Bonus:
 - Add sliders to show the values of the variables and the constraints.

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from pulp import *

def solve_optimization(profit_A_value=400):
    # Create the optimization problem (maximize profit)
    prob = LpProblem("Advanced_Production_Problem", LpMaximize)

    # Define Decision Variables
    A = LpVariable("A", lowBound=0, cat=LpInteger)  # must be integer
    B = LpVariable("B", lowBound=0)  # continous meters of plywood
    C = LpVariable("C", lowBound=0)  # continous meters of low quality plywood

    # Profit values
    profit_A = profit_A_value
    profit_B = 300
    profit_C = 200

    # Define Objective Function
    prob += profit_A*A + profit_B*B + profit_C*C, "Profit_Objective"

    # Define Constraints
    # Resource / Time Constraints
    prob += 0.3*A + 0.1*B + 0.1*C <= 1800, "Assembly_Hours"
    prob += 0.1*A + 0.08*B + 0.04*C <= 800, "Quality_Control_Hours"
    prob += 0.06*A + 0.04*B + 0.05*C <= 700, "Packaging_Hours"

    # Minimum Demand Constraints
    prob += A >= 100, "Min_Demand_A"
    prob += B >= 50,  "Min_Demand_B"
    prob += C >= 80,  "Min_Demand_C"

    # Solve the problem
    prob.solve(PULP_CBC_CMD(msg=False))  # CBC solver

    # Return results
    results = {
        "status": LpStatus[prob.status],
        "A": A.varValue,
        "B": B.varValue,
        "C": C.varValue,
        "total_profit": value(prob.objective)
    }

    return results

# Define slider for profit_A
profit_A_slider = widgets.FloatSlider(
    value=400,
    min=0,
    max=800,
    step=10,
    description='Profit A',
    continuous_update=False
)

# Output widget to display results
output = widgets.Output()

# Function to update results when slider changes
def update_results(change):
    with output:
        output.clear_output()
        results = solve_optimization(profit_A_slider.value)
        print(f"Status: {results['status']}")
        print(f"A = {results['A']}")
        print(f"B = {results['B']}")
        print(f"C = {results['C']}")
        print(f"Total profit = {results['total_profit']}")

# Connect the slider to the update function
profit_A_slider.observe(update_results, names='value')

# Display the slider and initial results
display(profit_A_slider)
display(output)

# Show initial results
update_results(None)

FloatSlider(value=400.0, continuous_update=False, description='Profit A', max=800.0, step=10.0)

Output()

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from pulp import *

def solve_optimization(profit_A_value=400, profit_B_value=300, profit_C_value=200):
    # Create the optimization problem (maximize profit)
    prob = LpProblem("Advanced_Production_Problem", LpMaximize)

    # Define Decision Variables
    A = LpVariable("A", lowBound=0, cat=LpInteger)  # must be integer
    B = LpVariable("B", lowBound=0)  # continous meters of plywood
    C = LpVariable("C", lowBound=0)  # continous meters of low quality plywood

    # Profit values
    profit_A = profit_A_value
    profit_B = profit_B_value
    profit_C = profit_C_value

    # Define Objective Function
    prob += profit_A*A + profit_B*B + profit_C*C, "Profit_Objective"

    # Define Constraints
    # Resource / Time Constraints
    prob += 0.3*A + 0.1*B + 0.1*C <= 1800, "Assembly_Hours"
    prob += 0.1*A + 0.08*B + 0.04*C <= 800, "Quality_Control_Hours"
    prob += 0.06*A + 0.04*B + 0.05*C <= 700, "Packaging_Hours"

    # Minimum Demand Constraints
    prob += A >= 100, "Min_Demand_A"
    prob += B >= 50,  "Min_Demand_B"
    prob += C >= 80,  "Min_Demand_C"

    # Solve the problem
    prob.solve(PULP_CBC_CMD(msg=False))  # CBC solver

    # Return results
    results = {
        "status": LpStatus[prob.status],
        "A": A.varValue,
        "B": B.varValue,
        "C": C.varValue,
        "total_profit": value(prob.objective)
    }

    return results

# Define slider for profit_A
profit_A_slider = widgets.FloatSlider(
    value=400,
    min=0,
    max=800,
    step=10,
    description='Profit A',
    continuous_update=False
)

profit_B_slider = widgets.FloatSlider(
    value=300,
    min=0,
    max=800,
    step=10,
    description='Profit B',
    continuous_update=False
)

profit_C_slider = widgets.FloatSlider(
    value=200,
    min=0,
    max=800,
    step=10,
    description='Profit C',
    continuous_update=False
)

# Output widget to display results
output = widgets.Output()

# Function to update results when slider changes
def update_results(change):
    with output:
        output.clear_output()
        results = solve_optimization(profit_A_slider.value, profit_B_slider.value, profit_C_slider.value)
        print(f"Status: {results['status']}")
        print(f"A = {results['A']}")
        print(f"B = {results['B']}")
        print(f"C = {results['C']}")
        print(f"Total profit = {results['total_profit']}")

# Connect the slider to the update function
profit_A_slider.observe(update_results, names='value')
profit_B_slider.observe(update_results, names='value')
profit_C_slider.observe(update_results, names='value')

# Display the slider and initial results
display(profit_A_slider)
display(profit_B_slider)
display(profit_C_slider)
display(output)

# Show initial results
update_results(None)

FloatSlider(value=400.0, continuous_update=False, description='Profit A', max=800.0, step=10.0)

FloatSlider(value=300.0, continuous_update=False, description='Profit B', max=800.0, step=10.0)

FloatSlider(value=200.0, continuous_update=False, description='Profit C', max=800.0, step=10.0)

Output()